In [1]:
import pandas as pd

from datetime import datetime
from _lib.data_preparation import remove_substandard_trips, df_calc_basic, df_join_generic_with_gps, read_gpx, calc_context
from _lib.data_preparation_help import val2year, val2zip, val2utf8
from _lib.settings import DATA_AFTER_PREPARATION_DIR

# FR Amiens

In [2]:
from _lib.settings import DATA_ORIGIN_AMIENS_DIR


SHORT_NAME = 'ami'

### 2016

In [3]:
df_ami16 = pd.read_csv(f'{DATA_ORIGIN_AMIENS_DIR}/detail_2016.csv', encoding='windows-1250')

print('Shape before: ', df_ami16.shape)

''' Column names normalization '''
df_ami16.columns = [cname.replace(' ', '').lower() for cname in df_ami16.columns]

''' Column data normalization '''
df_ami16['tripid'] = SHORT_NAME + df_ami16['tripid'].astype(str).replace(' ', '')

df_ami16['timestamp'] = df_ami16['timestamp'].apply(lambda x: datetime.fromtimestamp(float(x)).timestamp() )

df_ami16 = df_ami16.astype({'latitude': 'float', 'longitude': 'float'})

df_ami16 = remove_substandard_trips(df_ami16)
df_ami16 = df_calc_basic(df_ami16)

''' Removing points with 0 distance passed '''
df_ami16 = df_ami16[(df_ami16['distance'] != 0) | (df_ami16['end']) | (df_ami16['start'])]

print('Shape after: ', df_ami16.shape)

/tmp/ipykernel_2780/3928128295.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ami16 = pd.read_csv(f'{DATA_ORIGIN_AMIENS_DIR}/detail_2016.csv', encoding='windows-1250')


Shape before:  (424910, 8)
Removed 0 substandard trips.


duration: 100%|██████████| 424910/424910 [00:01<00:00, 354152.73it/s]


Shape after:  (305314, 13)


In [4]:
df_ami16_generic = pd.read_csv(f'{DATA_ORIGIN_AMIENS_DIR}/generic_2016.csv', encoding='windows-1250')

''' Column names normalization '''
df_ami16_generic.columns = [cname.replace(' ', '').lower() for cname in df_ami16_generic.columns]

df_ami16_generic['tripid'] = SHORT_NAME + df_ami16_generic['tripid'].apply(lambda x: x.replace(' ', ''))
df_ami16_generic['distance'] = df_ami16_generic['distance'].astype(float)
df_ami16_generic['valid'] = df_ami16_generic[df_ami16_generic['ecc'].notna()]['ecc'].apply(lambda x: False if x == 0 else True)
df_ami16_generic['avgspeed'] = df_ami16_generic['avgspeed'].astype(float)
df_ami16_generic['tracktype'] = df_ami16_generic[df_ami16_generic['tracktype'].notna()]['tracktype'].apply(val2utf8)
df_ami16_generic['male'] = df_ami16_generic[df_ami16_generic['sex'].notna()]['sex'].apply(lambda x: True if str(x).lower() == 'm' else (False if str(x).lower() == 'f' else float('nan')))
df_ami16_generic['yearofbirth'] = df_ami16_generic['year'].apply(val2year)
df_ami16_generic['profession'] = df_ami16_generic[df_ami16_generic['profession'].notna()]['profession'].apply(val2utf8)
df_ami16_generic['frequentuser'] = df_ami16_generic[df_ami16_generic['frequentuser'].notna()]['frequentuser'].apply(lambda x: False if x.lower() in ['no', 'non'] else False)
df_ami16_generic['zip'] = df_ami16_generic[df_ami16_generic['zip'].notna()]['zip'].apply(val2zip)
df_ami16_generic['source'] = df_ami16_generic[df_ami16_generic['source'].notna()]['source'].apply(val2utf8)
df_ami16_generic['typeofbike'] = df_ami16_generic[df_ami16_generic['typeofbike'].notna()]['typeofbike'].apply(val2utf8)
df_ami16_generic['typeoftrip'] = df_ami16_generic[df_ami16_generic['tipeoftrip'].notna()]['tipeoftrip'].apply(val2utf8)

df_ami16_generic.drop(['timestamp', 'startdt', 'ecc', 'sex', 'year', 'tipeoftrip', 'distance', 'avgspeed'], axis=1, inplace=True)

''' Joinig generic data with gps data '''
print('Shape before: ', df_ami16_generic.shape)

df_ami16_generic = df_join_generic_with_gps(df_ami16_generic, df_ami16)

print('Shape after: ', df_ami16_generic.shape)

Shape before:  (2107, 11)
Shape after:  (1749, 18)


### 2017

In [5]:
df_ami17 = pd.read_csv(f'{DATA_ORIGIN_AMIENS_DIR}/detail_2017.csv', encoding='windows-1250', sep=';')

print('Shape before: ', df_ami17.shape)

''' Column names normalization '''
df_ami17.columns = [cname.replace(' ', '').lower() for cname in df_ami17.columns]

''' Column data normalization '''
df_ami17['tripid'] = SHORT_NAME + df_ami17['tripid'].astype(str).replace(' ', '')

df_ami17['timestamp'] = df_ami17['timestamp'].apply(lambda x: datetime.fromtimestamp(float(x)).timestamp() )
df_ami17['latitude'] = df_ami17['latitude'].str.replace(',', '.').astype(float)
df_ami17['longitude'] = df_ami17['longitude'].str.replace(',', '.').astype(float)
df_ami17['altitude'] = df_ami17['altitude'].astype(float)

df_ami17 = remove_substandard_trips(df_ami17)
df_ami17 = df_calc_basic(df_ami17)

''' Removing points with 0 distance passed '''
df_ami17 = df_ami17[(df_ami17['distance'] != 0) | (df_ami17['end']) | (df_ami17['start'])]

print('Shape after: ', df_ami17.shape)

Shape before:  (904274, 8)
Removed 54 substandard trips.


duration: 100%|██████████| 860377/860377 [00:02<00:00, 319711.91it/s]


Shape after:  (859306, 13)


In [6]:
df_ami17_generic = pd.read_csv(f'{DATA_ORIGIN_AMIENS_DIR}/generic_2017.csv', encoding='windows-1250', sep=';')

''' Column names normalization '''
df_ami17_generic.columns = [cname.replace(' ', '').lower() for cname in df_ami17_generic.columns]

''' Column data normalization '''
df_ami17_generic['tripid'] = SHORT_NAME + df_ami17_generic['tripid'].astype(str).replace(' ', '')
df_ami17_generic['avgspeed'] = df_ami17_generic['avgspeed'].str.replace(',', '.').astype(float)
df_ami17_generic['distance'] = df_ami17_generic['totallength'].str.replace(',', '.').astype(float)
df_ami17_generic['valid'] = df_ami17_generic['valid'].apply(lambda x: False if str(x).lower() == 'no' else True)
df_ami17_generic['male'] = df_ami17_generic[df_ami17_generic['sex'].notna()]['sex'].apply(lambda x: True if str(x).lower() == 'male' else (False if str(x).lower() == 'female' else float('nan')))
df_ami17_generic['yearofbirth'] = df_ami17_generic['yearofbirth'].apply(val2year)
df_ami17_generic['typeofbike'] = df_ami17_generic[df_ami17_generic['typeofbike'].notna()]['typeofbike'].apply(val2utf8)
df_ami17_generic['typeoftrip'] = df_ami17_generic[df_ami17_generic['typeoftrip'].notna()]['typeoftrip'].apply(val2utf8)

df_ami17_generic.drop(['uploaded', 'sex', 'timestamp', 'startdate', 'starttime', 'duration', 'maxspeed', 'totallength', 'lengthvalid', 'avgspeed', 'distance'], axis=1, inplace=True)

''' Joinig generic data with gps data '''
print('Shape before: ', df_ami17_generic.shape)

df_ami17_generic = df_join_generic_with_gps(df_ami17_generic, df_ami17)

print('Shape after: ', df_ami17_generic.shape)

Shape before:  (2350, 6)
Shape after:  (1621, 13)


### Removing overall columns & records

In [7]:
''' DETAIL '''

print('Shape before. 2016:', df_ami16.shape, '2017:', df_ami17.shape)

df_ami16 = df_ami16[['tripid', 'latitude', 'longitude', 'timestamp', 'start']]
df_ami16 = df_ami16[df_ami16['tripid'].isin(df_ami16_generic['tripid'].tolist())]

df_ami17 = df_ami17[['tripid', 'latitude', 'longitude', 'timestamp', 'start']]
df_ami17 = df_ami17[df_ami17['tripid'].isin(df_ami17_generic['tripid'].tolist())]

print('Shape after. 2016:', df_ami16.shape, '2017:', df_ami17.shape)

Shape before. 2016: (305314, 13) 2017: (859306, 13)
Shape after. 2016: (273082, 5) 2017: (762428, 5)


In [8]:
''' GENERIC '''

print('Shape before. 2015:', df_ami16_generic.shape, '2016:', df_ami17_generic.shape)

df_ami16_generic = df_ami16_generic[['tripid', 'speedmax', 'speedavg_excluding_time', 'speedavg_over_time', 'distance', 'startts', 'endts', 'tracktype', 'source', 'profession', 'male', 'frequentuser', 'zip', 'yearofbirth', 'valid']]

df_ami17_generic = df_ami17_generic[['tripid', 'speedmax', 'speedavg_excluding_time', 'speedavg_over_time', 'distance', 'startts', 'endts', 'typeofbike', 'typeoftrip', 'male', 'yearofbirth', 'valid']]

print('Shape before. 2015:', df_ami16_generic.shape, '2016:', df_ami17_generic.shape)

Shape before. 2015: (1749, 18) 2016: (1621, 13)
Shape before. 2015: (1749, 15) 2016: (1621, 12)


### Datasets concatenaton

In [9]:
df_ami = pd.concat([df_ami16, df_ami17], ignore_index=True)
df_ami.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035510 entries, 0 to 1035509
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   tripid     1035510 non-null  object 
 1   latitude   1035510 non-null  float64
 2   longitude  1035510 non-null  float64
 3   timestamp  1035510 non-null  float64
 4   start      1035510 non-null  bool   
dtypes: bool(1), float64(3), object(1)
memory usage: 32.6+ MB


In [10]:
df_ami_generic = pd.concat([df_ami16_generic, df_ami17_generic], ignore_index=True)
df_ami_generic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3370 entries, 0 to 3369
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tripid                   3370 non-null   object 
 1   speedmax                 3370 non-null   float64
 2   speedavg_excluding_time  3370 non-null   float64
 3   speedavg_over_time       3370 non-null   float64
 4   distance                 3370 non-null   float64
 5   startts                  3370 non-null   float64
 6   endts                    3370 non-null   float64
 7   tracktype                1749 non-null   object 
 8   source                   1749 non-null   object 
 9   profession               1292 non-null   object 
 10  male                     1849 non-null   object 
 11  frequentuser             1749 non-null   object 
 12  zip                      845 non-null    object 
 13  yearofbirth              1946 non-null   object 
 14  valid                   

### Saving operations

In [11]:
df_ami.to_csv(f'{DATA_AFTER_PREPARATION_DIR}/{SHORT_NAME}.csv', index=False, sep=';')
df_ami_generic.to_csv(f'{DATA_AFTER_PREPARATION_DIR}/{SHORT_NAME}_generic.csv', index=False, sep=';')

# PL Wroclaw

In [12]:
from tqdm import tqdm
from _lib.settings import DATA_ORIGIN_WROCLAW_DIR


SHORT_NAME = 'wro'

### 2015

In [13]:
df_wro15 = pd.read_csv(f'{DATA_ORIGIN_WROCLAW_DIR}/detail_2015.csv', encoding='windows-1250', skiprows=[9627453])

print('Shape before: ', df_wro15.shape)

''' Column names normalization '''
df_wro15.columns = [cname.replace(' ', '').lower() for cname in df_wro15.columns]

''' Column data normalization '''
df_wro15['tripid'] = SHORT_NAME + df_wro15['tripid'].astype(str).replace(' ', '')

df_wro15 = df_wro15.astype({'latitude': 'float', 'longitude': 'float'})
df_wro15 = remove_substandard_trips(df_wro15)

tqdm.pandas(desc='timestamp')
df_wro15['timestamp'] = df_wro15['timestamp'].progress_apply(lambda x: float('nan') if str(x).lower() in ['false', 'nan'] else datetime.fromtimestamp(float(x)).timestamp())

df_wro15 = remove_substandard_trips(df_wro15)
df_wro15 = df_calc_basic(df_wro15)

''' Removing points with 0 distance passed '''
df_wro15 = df_wro15[(df_wro15['distance'] != 0) | (df_wro15['end']) | (df_wro15['start'])]

print('Shape after: ', df_wro15.shape)

/tmp/ipykernel_2780/3404425056.py:1: DtypeWarning: Columns (1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_wro15 = pd.read_csv(f'{DATA_ORIGIN_WROCLAW_DIR}/detail_2015.csv', encoding='windows-1250', skiprows=[9627453])


Shape before:  (20931869, 9)
Removed 21 substandard trips.


timestamp: 100%|██████████| 20931208/20931208 [00:54<00:00, 382387.46it/s]


Removed 1 substandard trips.


duration: 100%|██████████| 20930880/20930880 [01:23<00:00, 250827.15it/s]


Shape after:  (16334748, 15)


In [14]:
df_wro15_generic = pd.read_csv(f'{DATA_ORIGIN_WROCLAW_DIR}/generic_2015.csv')

''' Column names normalization '''
df_wro15_generic.columns = [cname.replace(' ', '').lower() for cname in df_wro15_generic.columns]

''' Column data normalization '''
df_wro15_generic['tripid'] = SHORT_NAME + df_wro15_generic['tripid'].apply(lambda id: id.replace(' ', ''))
df_wro15_generic['distance'] = df_wro15_generic['distance'].astype(float)
df_wro15_generic['valid'] = df_wro15_generic[df_wro15_generic['ecc'].notna()]['ecc'].apply(lambda x: False if x == 0 else True)
df_wro15_generic['avgspeed'] = df_wro15_generic['avgspeed'].astype(float)
df_wro15_generic['tracktype'] = df_wro15_generic[df_wro15_generic['tracktype'].notna()]['tracktype'].apply(val2utf8)
df_wro15_generic['male'] = df_wro15_generic[df_wro15_generic['sex'].notna()]['sex'].apply(lambda x: True if str(x).lower() == 'm' else (False if str(x).lower() == 'f' else float('nan')))
df_wro15_generic['yearofbirth'] = df_wro15_generic['year'].apply(val2year)
df_wro15_generic['profession'] = df_wro15_generic[df_wro15_generic['profession'].notna()]['profession'].apply(val2utf8)
df_wro15_generic['frequentuser'] = df_wro15_generic[df_wro15_generic['frequentuser'].notna()]['frequentuser'].apply(lambda x: False if x.lower() == 'no' else True)
df_wro15_generic['zip'] = df_wro15_generic[df_wro15_generic['zip'].notna()]['zip'].apply(val2zip)
df_wro15_generic['source'] = df_wro15_generic[df_wro15_generic['source'].notna()]['source'].apply(val2utf8)

df_wro15_generic.drop(['timestamp', 'startdt', 'ecc', 'sex', 'year', 'distance', 'avgspeed'], axis=1, inplace=True)

''' Joinig generic data with gps data '''
print('Shape before: ', df_wro15_generic.shape)

df_wro15_generic = df_join_generic_with_gps(df_wro15_generic, df_wro15)

print('Shape after: ', df_wro15_generic.shape)

Shape before:  (49941, 9)
Shape after:  (49154, 16)


### 2016

In [15]:
df_wro16 = pd.read_csv(f'{DATA_ORIGIN_WROCLAW_DIR}/detail_2016.csv', encoding='windows-1250', skiprows=[11184484])

print('Shape before: ', df_wro16.shape)

''' Column names normalization '''
df_wro16.columns = [cname.replace(' ', '').lower() for cname in df_wro16.columns]

''' Column data normalization '''
df_wro16['tripid'] = SHORT_NAME + df_wro16['tripid'].astype(str).replace(' ', '')

df_wro16 = df_wro16.astype({'latitude': 'float', 'longitude': 'float'})
df_wro16 = remove_substandard_trips(df_wro16)

tqdm.pandas(desc='timestamp')
df_wro16['timestamp'] = df_wro16['timestamp'].progress_apply(lambda x: float('nan') if str(x).lower() in ['false', 'nan'] else datetime.fromtimestamp(float(x)).timestamp() )

df_wro16 = remove_substandard_trips(df_wro16)
df_wro16 = df_calc_basic(df_wro16)

''' Removing points with 0 distance passed '''
df_wro16 = df_wro16[(df_wro16['distance'] != 0) | (df_wro16['end']) | (df_wro16['start'])]

print('Shape after: ', df_wro16.shape)

/tmp/ipykernel_2780/2463794659.py:1: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_wro16 = pd.read_csv(f'{DATA_ORIGIN_WROCLAW_DIR}/detail_2016.csv', encoding='windows-1250', skiprows=[11184484])


Shape before:  (22455984, 8)
Removed 2 substandard trips.


timestamp: 100%|██████████| 22453198/22453198 [01:06<00:00, 336442.11it/s]


Removed 0 substandard trips.


duration: 100%|██████████| 22453198/22453198 [01:06<00:00, 338635.87it/s]


Shape after:  (18122010, 14)


In [16]:
df_wro16_generic = pd.read_csv(f'{DATA_ORIGIN_WROCLAW_DIR}/generic_2016.csv')

''' Column names normalization '''
df_wro16_generic.columns = [cname.replace(' ', '').lower() for cname in df_wro16_generic.columns]

''' Column data normalization '''
df_wro16_generic['tripid'] = SHORT_NAME + df_wro16_generic['tripid'].apply(lambda x: x.replace(' ', ''))
df_wro16_generic['distance'] = df_wro16_generic['distance'].astype(float)
df_wro16_generic['valid'] = df_wro16_generic[df_wro16_generic['ecc'].notna()]['ecc'].apply(lambda x: False if x == 0 else True)
df_wro16_generic['avgspeed'] = df_wro16_generic['avgspeed'].astype(float)
df_wro16_generic['tracktype'] = df_wro16_generic[df_wro16_generic['tracktype'].notna()]['tracktype'].apply(val2utf8)
df_wro16_generic['male'] = df_wro16_generic[df_wro16_generic['sex'].notna()]['sex'].apply(lambda x: True if str(x).lower() == 'm' else (False if str(x).lower() == 'f' else float('nan')))
df_wro16_generic['yearofbirth'] = df_wro16_generic['year'].apply(val2year)
df_wro16_generic['profession'] = df_wro16_generic[df_wro16_generic['profession'].notna()]['profession'].apply(val2utf8)
df_wro16_generic['frequentuser'] = df_wro16_generic[df_wro16_generic['frequentuser'].notna()]['frequentuser'].apply(lambda x: False if x.lower() in ['no', 'nie'] else False)
df_wro16_generic['zip'] = df_wro16_generic[df_wro16_generic['zip'].notna()]['zip'].apply(val2zip)
df_wro16_generic['source'] = df_wro16_generic[df_wro16_generic['source'].notna()]['source'].apply(val2utf8)
df_wro16_generic['typeofbike'] = df_wro16_generic[df_wro16_generic['typeofbike'].notna()]['typeofbike'].apply(val2utf8)
df_wro16_generic['typeoftrip'] = df_wro16_generic[df_wro16_generic['tipeoftrip'].notna()]['tipeoftrip'].apply(val2utf8)

df_wro16_generic.drop(['timestamp', 'startdt', 'ecc', 'sex', 'year', 'distance', 'avgspeed'], axis=1, inplace=True)

''' Joinig generic data with gps data '''
print('Shape before: ', df_wro16_generic.shape)

df_wro16_generic = df_join_generic_with_gps(df_wro16_generic, df_wro16)

print('Shape after: ', df_wro16_generic.shape)

Shape before:  (42384, 12)
Shape after:  (41271, 19)


### Removing overall columns & records

In [17]:
''' DETAIL '''

print('Shape before. 2015:', df_wro15.shape, '2016:', df_wro16.shape)

df_wro15 = df_wro15[['tripid', 'latitude', 'longitude', 'timestamp', 'start']]
df_wro15 = df_wro15[df_wro15['tripid'].isin(df_wro15_generic['tripid'].tolist())]

df_wro16 = df_wro16[['tripid', 'latitude', 'longitude', 'timestamp', 'start']]
df_wro16 = df_wro16[df_wro16['tripid'].isin(df_wro16_generic['tripid'].tolist())]

print('Shape after. 2015:', df_wro15.shape, '2016:', df_wro16.shape)

Shape before. 2015: (16334748, 15) 2016: (18122010, 14)
Shape after. 2015: (16144800, 5) 2016: (17875093, 5)


In [18]:
''' GENERIC '''

print('Shape before. 2015:', df_wro15_generic.shape, '2016:', df_wro16_generic.shape)

df_wro15_generic = df_wro15_generic[['tripid', 'speedmax', 'speedavg_excluding_time', 'speedavg_over_time', 'distance', 'startts', 'endts', 'tracktype', 'source', 'profession', 'male', 'frequentuser', 'zip', 'yearofbirth', 'valid']]

df_wro16_generic = df_wro16_generic[['tripid', 'speedmax', 'speedavg_excluding_time', 'speedavg_over_time', 'distance', 'startts', 'endts', 'tracktype', 'typeofbike', 'typeoftrip', 'source', 'profession', 'male', 'frequentuser', 'zip', 'yearofbirth', 'valid']]

print('Shape before. 2015:', df_wro15_generic.shape, '2016:', df_wro16_generic.shape)

Shape before. 2015: (49154, 16) 2016: (41271, 19)
Shape before. 2015: (49154, 15) 2016: (41271, 17)


### Datasets concatenaton

In [19]:
df_wro = pd.concat([df_wro15, df_wro16], ignore_index=True)
df_wro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34019893 entries, 0 to 34019892
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   tripid     object 
 1   latitude   float64
 2   longitude  float64
 3   timestamp  float64
 4   start      bool   
dtypes: bool(1), float64(3), object(1)
memory usage: 1.0+ GB


In [20]:
df_wro_generic = pd.concat([df_wro15_generic, df_wro16_generic], ignore_index=True)
df_wro_generic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90425 entries, 0 to 90424
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tripid                   90425 non-null  object 
 1   speedmax                 90425 non-null  float64
 2   speedavg_excluding_time  90425 non-null  float64
 3   speedavg_over_time       90425 non-null  float64
 4   distance                 90425 non-null  float64
 5   startts                  90425 non-null  float64
 6   endts                    90425 non-null  float64
 7   tracktype                90425 non-null  object 
 8   source                   90425 non-null  object 
 9   profession               67403 non-null  object 
 10  male                     79218 non-null  object 
 11  frequentuser             90404 non-null  object 
 12  zip                      54714 non-null  object 
 13  yearofbirth              74237 non-null  object 
 14  valid                 

### Saving operations

In [21]:
df_wro.to_csv(f'{DATA_AFTER_PREPARATION_DIR}/{SHORT_NAME}.csv', index=False, sep=';')
df_wro_generic.to_csv(f'{DATA_AFTER_PREPARATION_DIR}/{SHORT_NAME}_generic.csv', index=False, sep=';')

# SE Orebro

In [22]:
from _lib.settings import DATA_ORIGIN_OREBRO_DIR


SHORT_NAME = 'ore'

### 2015

In [23]:
df_ore15, df_ore15_generic = read_gpx(f'{DATA_ORIGIN_OREBRO_DIR}/2015', SHORT_NAME)

df_ore15.shape, df_ore15_generic.shape

100%|██████████| 2156/2156 [01:03<00:00, 33.94it/s]


((648689, 5), (2156, 3))

In [24]:
print('Shape before: ', df_ore15.shape)

df_ore15 = remove_substandard_trips(df_ore15)
df_ore15 = df_calc_basic(df_ore15)

''' Removing points with 0 distance passed '''
df_ore15 = df_ore15[(df_ore15['distance'] != 0) | (df_ore15['end']) | (df_ore15['start'])]

print('Shape after: ', df_ore15.shape)

Shape before:  (648689, 5)
Removed 9 substandard trips.


duration: 100%|██████████| 648398/648398 [00:01<00:00, 358234.77it/s]


Shape after:  (512008, 12)


In [25]:
''' Joinig generic data with gps data '''
print('Shape before: ', df_ore15_generic.shape)

df_ore15_generic = df_join_generic_with_gps(df_ore15_generic, df_ore15)

print('Shape after: ', df_ore15_generic.shape)

Shape before:  (2156, 3)
Shape after:  (2110, 10)


### 2016

In [26]:
df_ore16, df_ore16_generic = read_gpx(f'{DATA_ORIGIN_OREBRO_DIR}/2016', SHORT_NAME)

df_ore16 = remove_substandard_trips(df_ore16)

df_ore16.shape, df_ore16_generic.shape

100%|██████████| 7992/7992 [00:03<00:00, 2089.06it/s]

Removed 0 substandard trips.


((0, 6), (0, 3))

In [27]:
# print('Shape before: ', df_ore16.shape)

# df_ore16 = remove_substandard_trips(df_ore16)
# df_ore16 = df_calc_basic(df_ore16)

# ''' Removing points with 0 distance passed '''
# df_ore16 = df_ore16[(df_ore16['distance'] != 0) | (df_ore16['end']) | (df_ore16['start'])]

# print('Shape after: ', df_ore16.shape)

In [28]:
# ''' Joinig generic data with gps data '''
# print('Shape before: ', df_ore16_generic.shape)

# df_ore16_generic = df_join_generic_with_gps(df_ore16_generic, df_ore16)

# print('Shape after: ', df_ore16_generic.shape)

### Removing overall columns & records

In [29]:
''' DETAIL '''

print('Shape before:', df_ore15.shape)

df_ore15 = df_ore15[['tripid', 'latitude', 'longitude', 'timestamp', 'start']]
df_ore15 = df_ore15[df_ore15['tripid'].isin(df_ore15_generic['tripid'].tolist())]

print('Shape after:', df_ore15.shape)

Shape before: (512008, 12)
Shape after: (509177, 5)


In [30]:
''' GENERIC '''

print('Shape before:', df_ore15_generic.shape)

df_ore15_generic = df_ore15_generic[['tripid', 'speedmax', 'speedavg_excluding_time', 'speedavg_over_time', 'distance', 'startts', 'endts', 'email']]

print('Shape after:', df_ore15_generic.shape)

Shape before: (2110, 10)
Shape after: (2110, 8)


In [31]:
df_ore15.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 509177 entries, 0 to 648397
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   tripid     509177 non-null  object 
 1   latitude   509177 non-null  float64
 2   longitude  509177 non-null  float64
 3   timestamp  509177 non-null  float64
 4   start      509177 non-null  bool   
dtypes: bool(1), float64(3), object(1)
memory usage: 19.9+ MB


In [32]:
df_ore15_generic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2110 entries, 0 to 2109
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tripid                   2110 non-null   object 
 1   speedmax                 2110 non-null   float64
 2   speedavg_excluding_time  2110 non-null   float64
 3   speedavg_over_time       2110 non-null   float64
 4   distance                 2110 non-null   float64
 5   startts                  2110 non-null   float64
 6   endts                    2110 non-null   float64
 7   email                    2110 non-null   object 
dtypes: float64(6), object(2)
memory usage: 132.0+ KB


### Saving operations

In [33]:
df_ore15.to_csv(f'{DATA_AFTER_PREPARATION_DIR}/{SHORT_NAME}.csv', index=False, sep=';')
df_ore15_generic.to_csv(f'{DATA_AFTER_PREPARATION_DIR}/{SHORT_NAME}_generic.csv', index=False, sep=';')

# DE Oldenburg

In [34]:
import os
import numpy as np

from tqdm import tqdm
from _lib.settings import DATA_ORIGIN_OLDENBURG_DIR


SHORT_NAME = 'old'

### Reading CSV files 2020

In [35]:
fpaths = []
for (dirpath, dirnames, filenames) in os.walk(f'{DATA_ORIGIN_OLDENBURG_DIR}/2020'):
    for file in filenames:
        if file.endswith(".csv"):
            fpaths.append(os.path.join(dirpath, file))

id, lat, lon, ts = [], [], [], []

for fpath in tqdm(fpaths):
    tripid = SHORT_NAME + fpath[:-4].split('-')[-1]
    df_trip = pd.read_csv(fpath, sep=';')
    df_trip['timestamp'] = pd.to_datetime(df_trip['measured_date'])
    df_trip['timestamp'] = df_trip['timestamp'].apply(lambda x: datetime.timestamp(x))
    
    id = id + [tripid] * df_trip.shape[0]
    lat = lat + df_trip['latitude'].tolist()
    lon = lon + df_trip['longitude'].tolist()
    ts = ts + df_trip['timestamp'].tolist()

df_old = pd.DataFrame(np.array([id, lat, lon, ts]).T, columns=['tripid', 'latitude', 'longitude', 'timestamp'])
df_old = df_old.astype({'latitude': 'float', 'longitude': 'float', 'timestamp': 'float'})

df_old.shape

100%|██████████| 1181/1181 [01:37<00:00, 12.16it/s]


(908080, 4)

### Processing

In [36]:
print('Shape before: ', df_old.shape)

df_old = remove_substandard_trips(df_old)
df_old = df_calc_basic(df_old)

''' Removing points with 0 distance passed '''
df_old = df_old[(df_old['distance'] != 0) | (df_old['end']) | (df_old['start'])]

print('Shape after: ', df_old.shape)

Shape before:  (908080, 4)
Removed 146 substandard trips.


duration: 100%|██████████| 732216/732216 [00:02<00:00, 360947.74it/s]


Shape after:  (731660, 11)


In [37]:
df_old_generic = calc_context(df_old)

print('Shape before: ', df_old_generic.shape)

df_old_generic.drop_duplicates(subset=list(set(df_old_generic.columns.tolist()) - set(['startts', 'endts'])), keep='first', inplace=True)

df_old_generic = df_old_generic.reset_index(inplace=False)

print('Shape after: ', df_old_generic.shape)

Shape before:  (1034, 7)
Shape after:  (1034, 8)


### Removing overall columns & records

In [38]:
''' DETAIL '''

print('Shape before:', df_old.shape)

df_old = df_old[['tripid', 'latitude', 'longitude', 'timestamp', 'start']]
df_old = df_old[df_old['tripid'].isin(df_old_generic['tripid'].tolist())]

print('Shape after:', df_old.shape)

Shape before: (731660, 11)
Shape after: (731659, 5)


In [39]:
''' GENERIC '''

print('Shape before:', df_old_generic.shape)

df_old_generic = df_old_generic[['tripid', 'speedmax', 'speedavg_excluding_time', 'speedavg_over_time', 'distance', 'startts', 'endts']]

print('Shape after:', df_old_generic.shape)

Shape before: (1034, 8)
Shape after: (1034, 7)


In [40]:
df_old.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 731659 entries, 0 to 732215
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   tripid     731659 non-null  object 
 1   latitude   731659 non-null  float64
 2   longitude  731659 non-null  float64
 3   timestamp  731659 non-null  float64
 4   start      731659 non-null  bool   
dtypes: bool(1), float64(3), object(1)
memory usage: 28.6+ MB


In [41]:
df_old_generic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1034 entries, 0 to 1033
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tripid                   1034 non-null   object 
 1   speedmax                 1034 non-null   float64
 2   speedavg_excluding_time  1034 non-null   float64
 3   speedavg_over_time       1034 non-null   float64
 4   distance                 1034 non-null   float64
 5   startts                  1034 non-null   float64
 6   endts                    1034 non-null   float64
dtypes: float64(6), object(1)
memory usage: 56.7+ KB


### Saving operations

In [42]:
df_old.to_csv(f'{DATA_AFTER_PREPARATION_DIR}/{SHORT_NAME}.csv', index=False, sep=';')
df_old_generic.to_csv(f'{DATA_AFTER_PREPARATION_DIR}/{SHORT_NAME}_generic.csv', index=False, sep=';')

# DE Berlin

In [43]:
import os
import numpy as np

from tqdm import tqdm
from _lib.settings import DATA_ORIGIN_BERLIN_DIR


SHORT_NAME = 'ber'

### Reading files 2020 - 2021

In [44]:
fpaths = []
for (dirpath, dirnames, filenames) in os.walk(f'{DATA_ORIGIN_BERLIN_DIR}/2020_2021'):
    for file in filenames:
        if file.endswith(""):
            fpaths.append(os.path.join(dirpath, file))

id, lat, lon, ts = [], [], [], []

for fpath in tqdm(fpaths):
    tripid = SHORT_NAME + fpath.split('/')[-1].split('-')[-1]
    with open(fpath) as fr:
        Lines = fr.readlines()
        begin = False
        for line in Lines:
            if not begin:
                begin = 'lat,lon,X,Y,Z,timeStamp' in line
            else:
                lline = line.split(',')
                if lline[0] != '':
                    id.append(tripid)
                    lat.append(lline[0])
                    lon.append(lline[1])
                    ts.append(lline[5][:-3] + '.' + lline[5][-3:])

df_ber = pd.DataFrame(np.array([id, lat, lon, ts]).T, columns=['tripid', 'latitude', 'longitude', 'timestamp'])
df_ber = df_ber.astype({'latitude': 'float', 'longitude': 'float', 'timestamp': 'float'})

df_ber.shape

100%|██████████| 22931/22931 [03:04<00:00, 124.00it/s]


(12977703, 4)

In [45]:
print('Shape before: ', df_ber.shape)

df_ber = remove_substandard_trips(df_ber)
df_ber = df_calc_basic(df_ber)

''' Removing points with 0 distance passed '''
df_ber = df_ber[(df_ber['distance'] != 0) | (df_ber['end']) | (df_ber['start'])]

print('Shape after: ', df_ber.shape)

Shape before:  (12977703, 4)
Removed 0 substandard trips.


duration: 100%|██████████| 12977703/12977703 [00:36<00:00, 360437.68it/s]


Shape after:  (10923799, 11)


In [46]:
df_ber_generic = calc_context(df_ber)

print('Shape before: ', df_ber_generic.shape)

df_ber_generic.drop_duplicates(subset=list(set(df_ber_generic.columns.tolist()) - set(['startts', 'endts'])), keep='first', inplace=True)

df_ber_generic = df_ber_generic.reset_index(inplace=False)

print('Shape after: ', df_ber_generic.shape)

Shape before:  (22886, 7)
Shape after:  (22544, 8)


### Removing overall columns & records

In [47]:
''' DETAIL '''

print('Shape before:', df_ber.shape)

df_ber = df_ber[['tripid', 'latitude', 'longitude', 'timestamp', 'start']]
df_ber = df_ber[df_ber['tripid'].isin(df_ber_generic['tripid'].tolist())]

print('Shape after:', df_ber.shape)

Shape before: (10923799, 11)
Shape after: (10734646, 5)


In [48]:
''' GENERIC '''

print('Shape before:', df_ber_generic.shape)

df_ber_generic = df_ber_generic[['tripid', 'speedmax', 'speedavg_excluding_time', 'speedavg_over_time', 'distance', 'startts', 'endts']]

print('Shape after:', df_ber_generic.shape)

Shape before: (22544, 8)
Shape after: (22544, 7)


In [49]:
df_ber.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10734646 entries, 0 to 12976110
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   tripid     object 
 1   latitude   float64
 2   longitude  float64
 3   timestamp  float64
 4   start      bool   
dtypes: bool(1), float64(3), object(1)
memory usage: 419.7+ MB


In [50]:
df_ber_generic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22544 entries, 0 to 22543
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tripid                   22544 non-null  object 
 1   speedmax                 22544 non-null  float64
 2   speedavg_excluding_time  22544 non-null  float64
 3   speedavg_over_time       22544 non-null  float64
 4   distance                 22544 non-null  float64
 5   startts                  22544 non-null  float64
 6   endts                    22544 non-null  float64
dtypes: float64(6), object(1)
memory usage: 1.2+ MB


### Saving operations

In [51]:
df_ber.to_csv(f'{DATA_AFTER_PREPARATION_DIR}/{SHORT_NAME}.csv', index=False, sep=';')
df_ber_generic.to_csv(f'{DATA_AFTER_PREPARATION_DIR}/{SHORT_NAME}_generic.csv', index=False, sep=';')

# PL Gdansk

In [52]:
from tqdm import tqdm
from _lib.settings import DATA_ORIGIN_GDANSK_DIR


SHORT_NAME = 'gda'

### 2015

In [53]:
df_gda15 = pd.read_csv(f'{DATA_ORIGIN_GDANSK_DIR}/detail_2015.csv', encoding='windows-1250')

print('Shape before: ', df_gda15.shape)

''' Column names normalization '''
df_gda15.columns = [cname.replace(' ', '').lower() for cname in df_gda15.columns]

''' Column data normalization '''
df_gda15['tripid'] = SHORT_NAME + df_gda15['tripid'].astype(str).replace(' ', '')

df_gda15 = df_gda15.astype({'latitude': 'float', 'longitude': 'float'})
df_gda15 = remove_substandard_trips(df_gda15)

tqdm.pandas(desc='timestamp')
df_gda15['timestamp'] = df_gda15['timestamp'].progress_apply(lambda x: float('nan') if str(x).lower() in ['false', 'nan'] else datetime.fromtimestamp(float(x)).timestamp() )

df_gda15 = remove_substandard_trips(df_gda15)
df_gda15 = df_calc_basic(df_gda15)

''' Removing points with 0 distance passed '''
df_gda15 = df_gda15[(df_gda15['distance'] != 0) | (df_gda15['end']) | (df_gda15['start'])]

print('Shape after: ', df_gda15.shape)

/tmp/ipykernel_2780/1703542522.py:1: DtypeWarning: Columns (1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_gda15 = pd.read_csv(f'{DATA_ORIGIN_GDANSK_DIR}/detail_2015.csv', encoding='windows-1250')


Shape before:  (26762382, 9)
Removed 21 substandard trips.


timestamp: 100%|██████████| 26759422/26759422 [01:01<00:00, 434617.74it/s]


Removed 2 substandard trips.


duration: 100%|██████████| 26754447/26754447 [01:14<00:00, 357671.44it/s]


Shape after:  (22495726, 15)


In [54]:
df_gda15_generic = pd.read_csv(f'{DATA_ORIGIN_GDANSK_DIR}/generic_2015.csv')

''' Column names normalization '''
df_gda15_generic.columns = [cname.replace(' ', '').lower() for cname in df_gda15_generic.columns]

''' Column data normalization '''
df_gda15_generic['tripid'] = SHORT_NAME + df_gda15_generic['tripid'].apply(lambda id: id.replace(' ', ''))
df_gda15_generic['distance'] = df_gda15_generic['distance'].astype(float)
df_gda15_generic['valid'] = df_gda15_generic[df_gda15_generic['ecc'].notna()]['ecc'].apply(lambda x: False if x == 0 else True)
df_gda15_generic['avgspeed'] = df_gda15_generic['avgspeed'].astype(float)
df_gda15_generic['tracktype'] = df_gda15_generic[df_gda15_generic['tracktype'].notna()]['tracktype'].apply(val2utf8)
df_gda15_generic['male'] = df_gda15_generic[df_gda15_generic['sex'].notna()]['sex'].apply(lambda x: True if str(x).lower() == 'm' else (False if str(x).lower() == 'f' else float('nan')))
df_gda15_generic['yearofbirth'] = df_gda15_generic['year'].apply(val2year)
df_gda15_generic['profession'] = df_gda15_generic[df_gda15_generic['profession'].notna()]['profession'].apply(val2utf8)
df_gda15_generic['frequentuser'] = df_gda15_generic[df_gda15_generic['frequentuser'].notna()]['frequentuser'].apply(lambda x: False if x.lower() == 'no' else True)
df_gda15_generic['zip'] = df_gda15_generic[df_gda15_generic['zip'].notna()]['zip'].apply(val2zip)
df_gda15_generic['source'] = df_gda15_generic[df_gda15_generic['source'].notna()]['source'].apply(val2utf8)

df_gda15_generic.drop(['timestamp', 'startdt', 'ecc', 'sex', 'year', 'distance', 'avgspeed'], axis=1, inplace=True)

''' Joinig generic data with gps data '''
print('Shape before: ', df_gda15_generic.shape)

df_gda15_generic = df_join_generic_with_gps(df_gda15_generic, df_gda15)

print('Shape after: ', df_gda15_generic.shape)

Shape before:  (55279, 9)
Shape after:  (54244, 16)


### 2016

In [55]:
df_gda16 = pd.read_csv(f'{DATA_ORIGIN_GDANSK_DIR}/detail_2016.csv', encoding='windows-1250', skiprows=[11184484])

print('Shape before: ', df_gda16.shape)

''' Column names normalization '''
df_gda16.columns = [cname.replace(' ', '').lower() for cname in df_gda16.columns]

''' Column data normalization '''
df_gda16['tripid'] = SHORT_NAME + df_gda16['tripid'].astype(str).replace(' ', '')

df_gda16 = df_gda16.astype({'latitude': 'float', 'longitude': 'float'})
df_gda16 = remove_substandard_trips(df_gda16)

tqdm.pandas(desc='timestamp')
df_gda16['timestamp'] = df_gda16['timestamp'].progress_apply(lambda x: float('nan') if str(x).lower() in ['false', 'nan'] else datetime.fromtimestamp(float(x)).timestamp() )

df_gda16 = remove_substandard_trips(df_gda16)
df_gda16 = df_calc_basic(df_gda16)

''' Removing points with 0 distance passed '''
df_gda16 = df_gda16[(df_gda16['distance'] != 0) | (df_gda16['end']) | (df_gda16['start'])]

print('Shape after: ', df_gda16.shape)

/tmp/ipykernel_2780/630622996.py:1: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_gda16 = pd.read_csv(f'{DATA_ORIGIN_GDANSK_DIR}/detail_2016.csv', encoding='windows-1250', skiprows=[11184484])


Shape before:  (48208935, 8)
Removed 1 substandard trips.


timestamp: 100%|██████████| 48207531/48207531 [01:56<00:00, 415246.75it/s]


Removed 1 substandard trips.


duration: 100%|██████████| 48204011/48204011 [02:20<00:00, 343815.32it/s]


Shape after:  (40168472, 14)


In [56]:
df_gda16_generic = pd.read_csv(f'{DATA_ORIGIN_GDANSK_DIR}/generic_2016.csv')

''' Column names normalization '''
df_gda16_generic.columns = [cname.replace(' ', '').lower() for cname in df_gda16_generic.columns]

''' Column data normalization '''
df_gda16_generic['tripid'] = SHORT_NAME + df_gda16_generic['tripid'].apply(lambda x: x.replace(' ', ''))
df_gda16_generic['distance'] = df_gda16_generic['distance'].astype(float)
df_gda16_generic['valid'] = df_gda16_generic[df_gda16_generic['ecc'].notna()]['ecc'].apply(lambda x: False if x == 0 else True)
df_gda16_generic['avgspeed'] = df_gda16_generic['avgspeed'].astype(float)
df_gda16_generic['tracktype'] = df_gda16_generic[df_gda16_generic['tracktype'].notna()]['tracktype'].apply(val2utf8)
df_gda16_generic['male'] = df_gda16_generic[df_gda16_generic['sex'].notna()]['sex'].apply(lambda x: True if str(x).lower() == 'm' else (False if str(x).lower() == 'f' else float('nan')))
df_gda16_generic['yearofbirth'] = df_gda16_generic['year'].apply(val2year)
df_gda16_generic['profession'] = df_gda16_generic[df_gda16_generic['profession'].notna()]['profession'].apply(val2utf8)
df_gda16_generic['frequentuser'] = df_gda16_generic[df_gda16_generic['frequentuser'].notna()]['frequentuser'].apply(lambda x: False if x.lower() in ['no', 'nie'] else False)
df_gda16_generic['zip'] = df_gda16_generic[df_gda16_generic['zip'].notna()]['zip'].apply(val2zip)
df_gda16_generic['source'] = df_gda16_generic[df_gda16_generic['source'].notna()]['source'].apply(val2utf8)
df_gda16_generic['typeofbike'] = df_gda16_generic[df_gda16_generic['typeofbike'].notna()]['typeofbike'].apply(val2utf8)
df_gda16_generic['typeoftrip'] = df_gda16_generic[df_gda16_generic['tipeoftrip'].notna()]['tipeoftrip'].apply(val2utf8)

df_gda16_generic.drop(['timestamp', 'startdt', 'ecc', 'sex', 'year', 'distance', 'avgspeed'], axis=1, inplace=True)

''' Joinig generic data with gps data '''
print('Shape before: ', df_gda16_generic.shape)

df_gda16_generic = df_join_generic_with_gps(df_gda16_generic, df_gda16)

print('Shape after: ', df_gda16_generic.shape)

Shape before:  (88992, 12)
Shape after:  (85872, 19)


### Removing overall columns & records

In [57]:
''' DETAIL '''

print('Shape before. 2015:', df_gda15.shape, '2016:', df_gda16.shape)

df_gda15 = df_gda15[['tripid', 'latitude', 'longitude', 'timestamp', 'start']]
df_gda15 = df_gda15[df_gda15['tripid'].isin(df_gda15_generic['tripid'].tolist())]

df_gda16 = df_gda16[['tripid', 'latitude', 'longitude', 'timestamp', 'start']]
df_gda16 = df_gda16[df_gda16['tripid'].isin(df_gda16_generic['tripid'].tolist())]

print('Shape after. 2015:', df_gda15.shape, '2016:', df_gda16.shape)

Shape before. 2015: (22495726, 15) 2016: (40168472, 14)
Shape after. 2015: (22326294, 5) 2016: (39343640, 5)


In [58]:
''' GENERIC '''

print('Shape before. 2015:', df_gda15_generic.shape, '2016:', df_gda16_generic.shape)

df_gda15_generic = df_gda15_generic[['tripid', 'speedmax', 'speedavg_excluding_time', 'speedavg_over_time', 'distance', 'startts', 'endts', 'tracktype', 'source', 'profession', 'male', 'frequentuser', 'zip', 'yearofbirth', 'valid']]

df_gda16_generic = df_gda16_generic[['tripid', 'speedmax', 'speedavg_excluding_time', 'speedavg_over_time', 'distance', 'startts', 'endts', 'tracktype', 'typeofbike', 'typeoftrip', 'source', 'profession', 'male', 'frequentuser', 'zip', 'yearofbirth', 'valid']]

print('Shape before. 2015:', df_gda15_generic.shape, '2016:', df_gda16_generic.shape)

Shape before. 2015: (54244, 16) 2016: (85872, 19)
Shape before. 2015: (54244, 15) 2016: (85872, 17)


### Datasets concatenaton

In [59]:
df_gda = pd.concat([df_gda15, df_gda16], ignore_index=True)
df_gda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61669934 entries, 0 to 61669933
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   tripid     object 
 1   latitude   float64
 2   longitude  float64
 3   timestamp  float64
 4   start      bool   
dtypes: bool(1), float64(3), object(1)
memory usage: 1.9+ GB


In [60]:
df_gda_generic = pd.concat([df_gda15_generic, df_gda16_generic], ignore_index=True)
df_gda_generic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140116 entries, 0 to 140115
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   tripid                   140116 non-null  object 
 1   speedmax                 140116 non-null  float64
 2   speedavg_excluding_time  140116 non-null  float64
 3   speedavg_over_time       140116 non-null  float64
 4   distance                 140116 non-null  float64
 5   startts                  140116 non-null  float64
 6   endts                    140116 non-null  float64
 7   tracktype                140116 non-null  object 
 8   source                   140114 non-null  object 
 9   profession               104634 non-null  object 
 10  male                     120196 non-null  object 
 11  frequentuser             140116 non-null  bool   
 12  zip                      81446 non-null   object 
 13  yearofbirth              118068 non-null  object 
 14  vali

### Saving operations

In [61]:
df_gda.to_csv(f'{DATA_AFTER_PREPARATION_DIR}/{SHORT_NAME}.csv', index=False, sep=';')
df_gda_generic.to_csv(f'{DATA_AFTER_PREPARATION_DIR}/{SHORT_NAME}_generic.csv', index=False, sep=';')

# SW Sodertalie

In [64]:
from tqdm import tqdm
from _lib.settings import DATA_ORIGIN_SODERTALIE_DIR


SHORT_NAME = 'sod'

In [65]:
df_sod = pd.read_csv(f'{DATA_ORIGIN_SODERTALIE_DIR}/sodertalje_detail.csv')

print('Shape before: ', df_sod.shape)

''' Column names normalization '''
df_sod.columns = [cname.replace(' ', '').lower() for cname in df_sod.columns]

''' Column data normalization '''
df_sod['tripid'] = SHORT_NAME + df_sod['tripid'].astype(str).replace(' ', '')

tqdm.pandas(desc='timestamp')
df_sod['timestamp'] = df_sod['timestamp'].apply(lambda x: datetime.fromtimestamp(float(x)).timestamp() )

df_sod.drop(['altitude', 'distance', 'speed', 'type'], axis=1, inplace=True)

df_sod = df_sod.astype({'latitude': 'float', 'longitude': 'float'})

df_sod = remove_substandard_trips(df_sod)
df_sod = df_calc_basic(df_sod)

''' Removing points with 0 distance passed '''
df_sod = df_sod[(df_sod['distance'] != 0) | (df_sod['end']) | (df_sod['start'])]

print('Shape after: ', df_sod.shape)

Shape before:  (594498, 8)
Removed 0 substandard trips.


duration: 100%|██████████| 594498/594498 [00:01<00:00, 334384.25it/s]


Shape after:  (457312, 11)


In [66]:
df_sod_generic = calc_context(df_sod)

print('Shape before: ', df_sod_generic.shape)

df_sod_generic.drop_duplicates(subset=list(set(df_sod_generic.columns.tolist()) - set(['startts', 'endts'])), keep='first', inplace=True)

df_sod_generic = df_sod_generic.reset_index(inplace=False)

print('Shape after: ', df_sod_generic.shape)

Shape before:  (1487, 7)
Shape after:  (1466, 8)


### Removing overall columns & records

In [67]:
''' DETAIL '''

print('Shape before:', df_sod.shape)

df_sod = df_sod[['tripid', 'latitude', 'longitude', 'timestamp', 'start']]
df_sod = df_sod[df_sod['tripid'].isin(df_sod_generic['tripid'].tolist())]

print('Shape after:', df_sod.shape)

Shape before: (457312, 11)
Shape after: (455007, 5)


In [68]:
''' GENERIC '''

print('Shape before:', df_sod_generic.shape)

df_sod_generic = df_sod_generic[['tripid', 'speedmax', 'speedavg_excluding_time', 'speedavg_over_time', 'distance', 'startts', 'endts']]

print('Shape after:', df_sod_generic.shape)

Shape before: (1466, 8)
Shape after: (1466, 7)


In [69]:
df_sod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 455007 entries, 0 to 594497
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   tripid     455007 non-null  object 
 1   latitude   455007 non-null  float64
 2   longitude  455007 non-null  float64
 3   timestamp  455007 non-null  float64
 4   start      455007 non-null  bool   
dtypes: bool(1), float64(3), object(1)
memory usage: 17.8+ MB


In [70]:
df_sod_generic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1466 entries, 0 to 1465
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tripid                   1466 non-null   object 
 1   speedmax                 1466 non-null   float64
 2   speedavg_excluding_time  1466 non-null   float64
 3   speedavg_over_time       1466 non-null   float64
 4   distance                 1466 non-null   float64
 5   startts                  1466 non-null   float64
 6   endts                    1466 non-null   float64
dtypes: float64(6), object(1)
memory usage: 80.3+ KB


### Saving operations

In [71]:
df_sod.to_csv(f'{DATA_AFTER_PREPARATION_DIR}/{SHORT_NAME}.csv', index=False, sep=';')
df_sod_generic.to_csv(f'{DATA_AFTER_PREPARATION_DIR}/{SHORT_NAME}_generic.csv', index=False, sep=';')